In [ ]:
import os,sys
import torch
import numpy as np
import yaml
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=False)

# SET THIS VARIABLE TO THE LOCATION OF THE MLRECO REPOSITORY
SOFTWARE_DIR = '/home/twongjirad/working/larbys/lartpc_mlreco3d/'
sys.path.insert(0, SOFTWARE_DIR)

from mlreco.visualization import scatter_points, plotly_layout3d
from mlreco.visualization.gnn import scatter_clusters, network_topology, network_schematic
from mlreco.utils.ppn import uresnet_ppn_type_point_selector
from mlreco.utils.cluster.dense_cluster import fit_predict_np, gaussian_kernel
from mlreco.main_funcs import process_config, prepare
#from mlreco.utils.gnn.cluster import get_cluster_label
#from mlreco.utils.deghosting import adapt_labels_numpy as adapt_labels
from mlreco.visualization.gnn import network_topology

In [ ]:
# WE ARE USING LARCV1 (the submodule included in the ICDL repository)
import ROOT as rt
from larcv import larcv
from larlite import larlite
from larflow import larflow
from ROOT import larutil

In [ ]:
# SETUP GEOMETRY: FOR VISUALIZATION
DETECTOR = "uboone"

if DETECTOR == "icarus":
    detid = larlite.geo.kICARUS
elif DETECTOR == "uboone":
    detid = larlite.geo.kMicroBooNE
elif DETECTOR == "sbnd":
    detid = larlite.geo.kSBND
    
larutil.LArUtilConfig.SetDetector(detid)

import lardly
from lardly.detectoroutline import get_tpc_boundary_plot

tpclines = get_tpc_boundary_plot()

In [ ]:
# SETUP DATA LOADERS
datapath="/home/twongjirad/working/larbys/icarus/icdl/larflow/larmatchnet/dataprep/test_weights.root"

cfg = """
iotool:
  batch_size: 1
  shuffle: False
  num_workers: 1
  collate_fn: CollateSparse
  dataset:
    name: LArCVDataset
    data_keys:
      - %s
    limit_num_files: 10
    nvoxel_limit: 1000000
    #event_list: '[6436, 562, 3802, 6175, 15256]'
    schema:
      input_data:
          parser: parse_sparse3d
          args:
              sparse_event_list:
                  - sparse3d_charge_plane0
                  - sparse3d_charge_plane1
                  - sparse3d_charge_plane2
      segment_label:
          parser: parse_sparse3d
          args:
              sparse_event_list:
                  - sparse3d_semantics_ghost
      segment_weights:
        parser: parse_sparse3d
        args:
          sparse_event_list:
            - sparse3d_semantics_weights
      cluster_label:
          parser: parse_sparse3d
          args:
              sparse_event_list:
                  - sparse3d_pcluster
      particles_label:
          parser: parse_particle_points
          args: 
              sparse_event: sparse3d_charge_plane2
              particle_event: particle_corrected
              include_point_tagging: True
"""%(datapath)

In [ ]:
cfg=yaml.load(cfg,Loader=yaml.Loader)
# pre-process configuration (checks + certain non-specified default settings)
process_config(cfg)
# prepare function configures necessary "handlers"
hs=prepare(cfg)

In [ ]:
data = next(hs.data_io_iter)
data.keys()
# input_data: [N,C]
# C = (i,j,k,b,feat1,feat2,...)
print('input_data: ',data['input_data'].shape) 
print('segment_label: ',data['segment_label'].shape)
print('cluster_label: ',data['cluster_label'].shape)
print('particles_label: ',data['particles_label'].shape)

In [ ]:
print(data.keys())
print(data['index'])

In [ ]:
# Get one event from the Batch
BATCH_INDEX = 0
print("select shape: ",np.sum(data['input_data'][:, 0] == BATCH_INDEX))
input_data = data['input_data'][ (data['input_data'][:, 0].astype(np.int32) == BATCH_INDEX),:]
segment_label = data['segment_label'][data['segment_label'][:, 0] == BATCH_INDEX, -1]
particles_label = data['particles_label'][data['particles_label'][:, 0] == BATCH_INDEX]
cluster_label = data['cluster_label'][data['cluster_label'][:, 0] == BATCH_INDEX]
segment_weights = data['segment_weights'][data['segment_weights'][:, 0] == BATCH_INDEX]
print("input_data.shape: ",input_data.shape)
print("segment_label.shape: ",segment_label.shape)
print("segment_weights.shape: ",segment_weights.shape)
print("particles_label.shape: ",particles_label.shape)
print("cluster_label.shape: ",cluster_label.shape)

In [ ]:
# CHARGE 
trace = []

ADC_MAX = 100.0

# ADD GHOSTS
trace+= scatter_points(input_data[segment_label==5],markersize=1,color='rgb(125,125,125)', 
                       cmin=0,cmax=ADC_MAX)
trace[-1].name = 'input_data (ghost points)'
# trace[-1].marker.colorscale='viridis'

# ADD ALL NON-GHOST POINTS
trace+= scatter_points(input_data[segment_label < 5],markersize=1,color=input_data[segment_label < 5, -2], cmin=0,cmax=ADC_MAX)
trace[-1].name = 'input_data (true noghost)'
trace[-1].marker.colorscale='viridis'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)

In [ ]:
# SEMANTIC LABELS

PLOT_GHOSTS = False

trace = []

if PLOT_GHOSTS:
    trace+= scatter_points(input_data,markersize=1,color=segment_label, cmin=0,cmax=5, colorscale=plotly.colors.qualitative.D3)
    trace[-1].name = 'Semantic segmentation labels (including ghost points)'
    trace[-1].marker.colorscale='viridis'


trace+= scatter_points(input_data[segment_label < 5],markersize=1,color=segment_label[segment_label < 5], cmin=0,cmax=5, colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Semantic segmentation labels (w/o ghost points)'
trace[-1].marker.colorscale='viridis'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)

In [ ]:
# SEMANTIC WEIGHTS

PLOT_GHOSTS = False

trace = []

if PLOT_GHOSTS:
    trace+= scatter_points(input_data,markersize=1,color=segment_label, cmin=0,cmax=5, colorscale=plotly.colors.qualitative.D3)
    trace[-1].name = 'Semantic segmentation labels (including ghost points)'
    trace[-1].marker.colorscale='viridis'


trace+= scatter_points(input_data[segment_label < 5],markersize=1,
                       color=segment_weights[segment_label < 5][:,-1], 
                       cmin=0,cmax=20, 
                       colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Semantic Weights (w/o ghost points)'
#trace[-1].marker.colorscale='viridis'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)

In [ ]:
# CLUSTER ID LABELS
cluster_trace = []
SKIP_NULL_IDs = [0]

cluster_ids = np.unique(cluster_label[:,-1])
print("cluster ids: ",cluster_ids)

for cid in cluster_ids:
    
    if cid in SKIP_NULL_IDs:
        continue
    
    rcolor = np.random.randint(0,255,3)
    zcolor = "rgb(%d,%d,%d)"%(rcolor[0],rcolor[1],rcolor[2])
    opa = 0.8
    if cid==-1:
        opa = 0.01
        zcolor="rgb(120,120,120,0.1)"
    plot = go.Scatter3d(
        x=cluster_label[cluster_label[:,-1]==cid,1],
        y=cluster_label[cluster_label[:,-1]==cid,2],
        z=cluster_label[cluster_label[:,-1]==cid,3],
        mode='markers',
        name="ID[%d]"%(cid),
        marker=dict(
            size=1,
            color=zcolor,
            opacity=opa)
    )
    cluster_trace.append(plot)

fig = go.Figure(data=cluster_trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)

In [ ]:
# PPN
ADC_MAX=100.0
PLOT_CLUSTERID = True
CHARGE_PLANE=2

trace = []
#print(data['particles_label'][:,4:])
print("particle_label: ",particles_label.shape)
print("label types: ",np.unique(particles_label[:,4]))

# split into keypoint types
track_starts  = particles_label[ (particles_label[:,4]==1)*(particles_label[:,6]==0) ]
track_ends    = particles_label[ (particles_label[:,4]==1)*(particles_label[:,6]==1) ]
shower_starts = particles_label[ (particles_label[:,4]==0) ]
michel_starts = particles_label[ (particles_label[:,4]==2) ]
delta_starts  = particles_label[ (particles_label[:,4]==3) ]
print("track_starts: ",track_starts.shape)
print("track_ends: ",track_ends.shape)
print("shower_starts: ",shower_starts.shape)
print("michel_starts: ",michel_starts.shape)
print("delta_starts: ",delta_starts.shape)

ppn_colors = {0:'rgb(200,0,0)',
              1:'rgb(0,200,0)',
              2:'rgb(0,0,200)',
              3:'rgb(0,250,200)',
              4:'rgb(250,200,0)'}
ppn_name = {0:"Track Start",
           1:"Track End",
           2:"Shower Start",
           3:"Delta Start",
           4:"Michel Start"}

for ippn,ppn_data in enumerate([track_starts,track_ends,shower_starts,delta_starts,michel_starts]):
    if ppn_data.shape[0]==0:
        continue
    plot = go.Scatter3d(
        x=ppn_data[:,1],
        y=ppn_data[:,2],
        z=ppn_data[:,3],
        mode='markers',
        name=ppn_name[ippn],
        marker=dict(
            size=5,
            color=ppn_colors[ippn],
            opacity=0.5)
    )
    trace.append( plot )

if not PLOT_CLUSTERID:
    # add (true) charge voxels
    trace+= scatter_points(input_data[segment_label < 5],
                           markersize=1,
                           color=input_data[segment_label < 5,4+CHARGE_PLANE], 
                           cmin=0,cmax=ADC_MAX)
    trace[-1].name = 'input_data (true noghost)'
    trace[-1].marker.colorscale='viridis'
else:
    trace += cluster_trace

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))
iplot(fig)
